# Cabecera de codigo para seleccionar el dispositivo a usar

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1" or "-1" if you want to use the cpu;
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

# Importacion de Librerias

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from shutil import copy2 as copy
import multiprocessing.pool
from six.moves import range

from shutil import rmtree as delete
from IPython.display import display, clear_output
from time import sleep
import numpy as np

# Organizando el DataSet

In [3]:
%cd /home/jefelitman/DataSets

if os.path.isdir("hmdb51"):
    %cd hmdb51
else:
    os.mkdir("hmdb51")

/home/jefelitman/DataSets


In [4]:
raw_ds_path = "/home/jefelitman/DataSets/HMDB51/frames"

In [5]:
dst_ds_path = "/home/jefelitman/DataSets/hmdb51"

In [6]:
len(sorted(os.listdir(raw_ds_path)))

6766

In [ ]:
sorted(os.listdir(raw_ds_path))[:10]

In [ ]:
#saber la cantidad de videos
elementos = 0
for clase in sorted(os.listdir(raw_ds_path)):
    elementos += len(sorted(os.listdir(os.path.join(raw_ds_path, clase))))
elementos

In [7]:
splits_path = "/home/jefelitman/splits"
files_path = sorted(os.listdir(splits_path))
len(files_path)

153

### Creacion de las carpetas de splits

In [8]:
for split in ["split_1","split_2","split_3"]:
    split_path = os.path.join(dst_ds_path,split)
    os.mkdir(split_path)
    for modo in ["train", "test"]:
        modo_path = os.path.join(split_path,modo)
        os.mkdir(modo_path)

### Colocando los archivos en los splits segun corresponda

In [9]:
for file in files_path:
    archivo = open(os.path.join(splits_path,file),"r")
    clase = file.split("_test_")[0]
    
    if file.endswith("1.txt"):
        split_path = os.path.join(dst_ds_path,"split_1")
    elif file.endswith("2.txt"):
        split_path = os.path.join(dst_ds_path,"split_2")
    elif file.endswith("3.txt"):
        split_path = os.path.join(dst_ds_path,"split_3")
    else:
        raise ValueError('Archivo invalido encontrado')
    
    for linea in archivo:
        video = linea.split(" ")[0][:-4]
        if linea.split(" ")[1] == '1':
            modo = "train"
        elif linea.split(" ")[1] == '2':
            modo = "test"
        elif linea.split(" ")[1] == '0':
            modo = None
        else:
            raise ValueError("Ha ocurrido un error con la seleccion de los modos "
                            "puede que el archivo este mal formateado: {}".format(linea.split(" ")[1]))
        if modo:
            dir_ori_path = os.path.join(raw_ds_path,video)
            if not os.path.isdir(os.path.join(split_path,modo,clase)):
                os.mkdir(os.path.join(split_path,modo,clase))

            dest = os.path.join(split_path,modo,clase,video)
            if not os.path.isdir(dest):
                os.mkdir(dest) #Creamos la carpeta del destino

            for frame in os.listdir(dir_ori_path):
                ori = os.path.join(dir_ori_path,frame)
                copy(ori,dest)

In [ ]:
#Funcion para comprobar que cada split haya quedado bien hecho
for file in files_path:
    archivo = list(open(os.path.join(splits_path,file),"r"))
    
    if file.endswith("1.txt"):
        split_path = os.path.join(dst_ds_path,"raw_split_1")
    elif file.endswith("2.txt"):
        split_path = os.path.join(dst_ds_path,"raw_split_2")
    elif file.endswith("3.txt"):
        split_path = os.path.join(dst_ds_path,"raw_split_3")
    else:
        raise ValueError('Archivo invalido encontrado')
    
    for linea in archivo:
        video = linea.split(" ")[0][:-4]
        if linea.split(" ")[1][:-1] == '1':
            modo = "train"
        elif linea.split(" ")[1][:-1] == '2':
            modo = "test"
        else:
            raise ValueError("Ha ocurrido un error con la seleccion de los modos "
                            "puede que el archivo este mal formateado: {}".format(linea.split(" ")[1]))
    
    elementos = 0
    for clase in sorted(os.listdir(mode_path)):
        videos_path = os.path.join(mode_path,clase)
        elementos += len(os.listdir(videos_path))
        
    if len(archivo) == elementos:
        print(True)
    else:
        print(False)

In [ ]:
#Funcion para eliminar carpetas dentro de los frames que correspondan al flujo o algo similar
from shutil import rmtree as delete
for video in sorted(os.listdir("/home/jefelitman/DataSets/UCF101/frames")):
    frames_path = os.path.join("/home/jefelitman/DataSets/UCF101/frames",video)
    for frame in sorted(os.listdir(frames_path)):
        file = os.path.join("/home/jefelitman/DataSets/UCF101/frames",video,frame)
        if os.path.isdir(file):
            delete(file,ignore_errors=True)

In [13]:
#Funcion para eliminar los archivos y carpetas que comiencen con un .
root = "/home/jefelitman/DataSets/hmdb51/split_3"
modos = sorted(os.listdir(root))
eliminados = 0
for modo in modos:
    modo_path = os.path.join(root, modo)
    if modo.startswith("."):
        if os.path.isdir(modo_path):
            delete(modo_path, ignore_errors = True)
        else:
            os.remove(modo_path)
        eliminados += 1
        break
    clases = sorted(os.listdir(modo_path))
    
    for clase in clases:
        clase_path = os.path.join(modo_path, clase)
        if clase.startswith("."):
            if os.path.isdir(clase_path):
                delete(clase_path, ignore_errors = True)
            else:
                os.remove(clase_path)
            eliminados += 1
            break
        videos = sorted(os.listdir(clase_path))

        for video in videos:
            video_path = os.path.join(clase_path,video)
            if video.startswith("."):
                if os.path.isdir(video_path):
                    delete(video_path, ignore_errors = True)
                else:
                    os.remove(video_path)
                eliminados += 1
                break

            frames = sorted(os.listdir(video_path))

            for frame in frames:
                frame_path = os.path.join(video_path,frame)
                if frame.startswith("."):
                    if os.path.isdir(frame_path):
                        delete(frame_path, ignore_errors = True)
                    else:
                        os.remove(frame_path)
                    eliminados += 1
                    break
eliminados

0

In [ ]:
#Funcion para revisar los frames en las carpetas de train, test de todos los splits
minframes = 200
mode_path = "/home/jefelitman/DataSets/ucf101/split_3/test"
for clase in sorted(os.listdir(mode_path)):
    clase_path = os.path.join(mode_path,clase)
    for video in sorted(os.listdir(clase_path)):
        video_path = os.path.join(clase_path,video)
        if len(os.listdir(video_path)) < minframes:
            minframes = len(os.listdir(video_path))
            minvideo = video_path

print(minframes)
print(minvideo)